# 🤖 Gemini Multi-Agent workshop - Type 1

This notebook sets up a hierarchical agent team using **CrewAI** and **Google Gemini 2.5 Flash**.

In [15]:
!pip install -q crewai crewai_tools langchain-google-genai
!pip install tavily-python
!pip install -U langchain-community duckduckgo-search ddgs

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.3/40.3 kB 1.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 161.7/161.7 kB 5.7 MB/s eta 0:00:00


In [24]:
import os

from crewai.tools import tool
from crewai import Agent, Task, Crew, Process, LLM
from crewai_tools import TavilySearchTool

from langchain_community.tools import DuckDuckGoSearchRun

from google.colab import userdata

In [25]:
os.environ["MODEL_ID"] = 'gemini/gemini-2.5-flash'
os.environ["GEMINI_API_KEY"] = userdata.get('GOOGLE_API_KEY')
os.environ["SERPER_API_KEY"] = userdata.get('SERPER_API_KEY')
os.environ["TAVILY_API_KEY"] = userdata.get('TAVILY_API_KEY')

In [30]:
VERBOSE = True

tools = dict()
tasks = []

In [39]:
# -----------------------------------------------------------
# Load and configure the Large Language Model (LLM) connector
# -----------------------------------------------------------
# This initializes a connection to a Gemini model using the
# model identifier stored in an environment variable.
#
# Environment variables are used so that model selection can
# be changed without modifying the code (useful for staging,
# production, or experimentation).
#
# The temperature parameter controls randomness:
# - Lower values (e.g., 0.2) → more deterministic responses
# - Higher values (e.g., 0.9) → more creative/diverse output
#
# Additional parameters below allow fine-grained control over:
# - response length
# - sampling behavior
# - reproducibility
# - safety limits
# -----------------------------------------------------------


tools['gemini_llm'] = LLM(
    model = os.environ["MODEL_ID"],   # Model name/version loaded securely from environment
    temperature = 0.7,                # Creativity level (balanced between accuracy and diversity)

    # --- Optional advanced generation controls ---

    max_tokens=1024,                  # Maximum number of tokens allowed in the response
    top_p = 0.9,                      # Nucleus sampling: restricts token selection to top probability mass
    top_k = 40,                       # Limits sampling to top-k most likely tokens
    frequency_penalty = 0.2,          # Reduces repetition of previously used words
    presence_penalty = 0.1,           # Encourages introducing new topics/content

    # --- Stability & runtime behavior ---

    timeout = 30,                     # Request timeout in seconds
    seed = 42,                        # Fixed seed for reproducible outputs (optional)
)

In [10]:
# -----------------------------------------------------------
# Initialize external research/search tool
# -----------------------------------------------------------
# This tool allows agents to perform real-time web searches.
# It extends the agents’ capabilities beyond the base LLM by
# giving access to fresh, external information.
#
# Useful for:
# - fact-checking
# - market research
# - trend analysis
# - retrieving up-to-date data
#
# Tools like this act as “sensors” for agents, enabling them
# to gather evidence before generating responses.
# -----------------------------------------------------------

tools['tavily'] = TavilySearchTool()

In [26]:
# -----------------------------------------------------------
# Register DuckDuckGo search tool
# -----------------------------------------------------------
# This adds a web search capability to the shared tools
# registry under the key 'ddg'.
#
# Tools stored in a dictionary can be dynamically assigned
# to agents or tasks later in the workflow.
#
# DuckDuckGo search provides lightweight, privacy-focused
# web results and is useful for:
# - quick lookups
# - news discovery
# - fact validation
# - supplemental research
# -----------------------------------------------------------


@tool('DuckDuckGoSearch')
def duckduckgo_search(search_query: str):
    """Search the web for information on a given topic using DuckDuckGo."""
    return DuckDuckGoSearchRun().run(search_query)

tools['ddg'] = duckduckgo_search

In [37]:
# -----------------------------------------------------------
# Register Calculator tool
# -----------------------------------------------------------
# This adds a mathematical computation capability to the
# shared tools registry under the key 'calc'.
#
# Calculator tools allow agents to perform precise numeric
# reasoning instead of approximating math with an LLM.
#
# Useful for:
# - arithmetic operations
# - financial calculations
# - statistics
# - unit conversions
# - engineering/scientific formulas
#
# This improves accuracy and prevents hallucinated math.
# -----------------------------------------------------------


@tool('Calculator')
def calculator(expression: str):
    """
    Evaluate a mathematical expression safely and return the result.
    The input should be a valid arithmetic expression.
    Example: "25 * 4 + 10 / 2"
    """
    return eval(expression)


tools['calc'] = calculator

In [20]:
# -----------------------------------------------------------
# Define Researcher Agent
# -----------------------------------------------------------
# This agent is responsible for gathering up-to-date,
# evidence-based information before content creation.
#
# Unlike the writer, this agent is tool-enabled and can
# perform live searches to retrieve current data.
#
# It acts as the intelligence-gathering component of the
# system, ensuring outputs are grounded in recent research.
# -----------------------------------------------------------


researcher_1 = Agent(
    role = 'Senior Research Analyst 1',
    # Defines the agent’s expertise and authority level.
    # Encourages analytical, fact-driven reasoning.

    goal = 'Find the latest breakthroughs in {topic}',
    # Primary mission: identify cutting-edge developments.
    # {topic} will be dynamically injected at runtime.

    backstory = 'You are an expert researcher with a focus on 2026 tech trends.',
    # Behavioral prompt shaping the agent’s mindset.
    # Emphasizes forward-looking, trend-aware research.

    tools = [tools['tavily']],
    # Grants the agent access to external search capabilities.
    # Tools extend the agent beyond static LLM knowledge.
    # The agent can call Tavily to fetch real-time web data.

    llm = tools['gemini_llm'],
    # Core language model used for reasoning and synthesis.

    verbose = True
    # Enables detailed execution tracing.
    # Useful for debugging research steps and tool usage.
)

In [27]:
# -----------------------------------------------------------
# Define Researcher Agent (DuckDuckGo-powered version)
# -----------------------------------------------------------
# This agent specializes in real-time research using an
# external search tool. It focuses on discovering cutting-
# edge developments and synthesizing current information.
#
# Compared to a static LLM-only agent, this researcher can
# actively query the web, making it suitable for trend
# analysis and time-sensitive insights.
# -----------------------------------------------------------


researcher_2 = Agent(
    role = 'Senior Research Analyst 2',
    # Establishes authority and analytical expertise.
    # Encourages evidence-driven reasoning and structured research.

    goal = 'Find the latest breakthroughs in {topic}',
    # Core mission of the agent.
    # {topic} will be dynamically injected during execution.

    backstory = 'You are an expert researcher with a focus on 2026 tech trends.',
    # Behavioral context guiding style and priorities.
    # Promotes forward-looking, innovation-focused research.

    tools = [tools['ddg']],
    # Grants access to the DuckDuckGo search tool.
    # Wrapped in a list because agents can support multiple tools.
    # Enables live external research during reasoning.

    llm = tools['gemini_llm'],
    # Language model used for interpretation and synthesis.
    # Retrieved from the shared tools/config registry.

    verbose = True
    # Enables detailed execution logs.
    # Useful for debugging agent decisions and tool calls.
)

In [40]:
# -----------------------------------------------------------
# Define Hybrid Research + Calculator Agent
# -----------------------------------------------------------
# This agent combines live research capabilities with
# deterministic numerical computation.
#
# It can both gather external information AND perform
# precise calculations, making it suitable for analytics,
# forecasting, benchmarking, and data-driven reporting.
#
# This is a hybrid intelligence agent:
#   research + reasoning + math
# -----------------------------------------------------------


researcher_calculator = Agent(
    role = 'Senior Research Analyst',
    # Establishes expert-level authority in research and analysis.
    # Encourages evidence-based reasoning and structured thinking.

    goal = 'Research breakthroughs and perform data calculations for {topic}.',
    # Dual mission:
    # 1) Discover cutting-edge developments
    # 2) Execute supporting numerical analysis
    # {topic} is dynamically injected at runtime.

    backstory = 'Expert analyst with a focus on 2026 tech trends and metrics.',
    # Behavioral context shaping tone and priorities.
    # Promotes forward-looking, metric-driven analysis.

    tools = [tools['calc']],
    # Grants access to multiple external tools:
    # - search_tool → live web research
    # - calculator → exact mathematical computation
    # Tool chaining enables richer reasoning workflows.

    llm = tools['gemini_llm'],
    # Core language model used for synthesis and interpretation.

    verbose = True,
    # Enables detailed execution tracing.
    # Useful for debugging tool calls and reasoning steps.

    allow_delegation = True
    # Allows the agent to delegate subtasks to other agents.
    # Required for hierarchical or multi-agent workflows.
)

In [41]:
# -----------------------------------------------------------
# Define AI agents for the multi-agent workflow
# -----------------------------------------------------------
# Each agent is configured with a professional role, a goal,
# and a behavioral backstory. These are not decorative —
# they act as structured prompts that guide how the model
# reasons, communicates, and makes decisions.
#
# Both agents share the same underlying LLM but behave
# differently due to their role configuration.
# -----------------------------------------------------------


# ---------------- Writer Agent ----------------
# Responsible for generating the final written report.
# This agent specializes in transforming technical research
# into clear, engaging, and actionable content.

writer = Agent(
    role = 'Technical Content Strategist',
    # Defines the agent’s professional identity and tone.
    # Influences writing style: analytical, structured, and persuasive.

    goal = 'Write a compelling report on {topic}',
    # Primary mission of the agent.
    # {topic} is a dynamic placeholder that will be injected at runtime.

    backstory = 'You translate complex research into simple, actionable insights.',
    # Behavioral guidance that shapes decision-making and style.
    # Encourages clarity, simplification, and audience-focused writing.

    llm = tools['gemini_llm'],
    # The language model engine powering the agent.
    # All reasoning and generation are performed using this model instance.

    verbose = VERBOSE
    # Enables detailed execution logs if VERBOSE = True.
    # Useful for debugging agent reasoning and prompt flow.
)


# ---------------- Manager Agent ----------------
# Oversees the workflow and ensures quality control.
# Acts like a supervisor coordinating team output.

manager = Agent(
    role = 'Project Manager',
    # Defines leadership-oriented behavior.
    # Encourages structured planning and evaluation.

    goal = 'Manage the team to deliver a perfect final report',
    # High-level coordination objective.
    # Focuses on outcomes, quality, and deadlines.

    backstory = 'You ensure quality and delegate tasks between the researcher and writer.',
    # Encourages delegation, review, and optimization.
    # Guides the agent to act as a workflow orchestrator.

    llm = tools['gemini_llm'],
    # Uses the same LLM backend, but behaves differently due to role prompts.

    allow_delegation = VERBOSE
    # Controls whether the manager can assign tasks to other agents.
    # When enabled, the manager can dynamically distribute work.
)


In [44]:
# Define Tasks
tasks.append(Task(
    description = 'Research the top 3 developments in {topic} for the year {year}.',
    expected_output = 'A detailed bulleted list of findings.',
    agent = researcher_1))

tasks.append(Task(
    description = 'Research the top 3 developments in {topic} for the year {year}.',
    expected_output = 'A detailed bulleted list of findings.',
    agent = researcher_2))

tasks.append(Task(
    description = 'Create a summary based on the research provided by the diferent researchers.',
    expected_output = 'A 5-paragraph executive summary.',
    agent = writer))

tasks.append(Task(
    description='Analyze the budget or numerical growth of {topic} in {year} using the calculator.',
    expected_output='A summary of the numerical data and calculations.',
    agent=researcher_2
))

tasks.append(Task(
    description='Analyze the budget or numerical growth of {topic} in {year} using the calculator.',
    expected_output='A summary of the numerical data and calculations.',
    agent=researcher_calculator
))

math_task = Task(
    description="""
        Calculate the mathematical operation: {operation}.
    """,
    expected_output="A brief report with the specific calculated growth figures.",
    agent=researcher_2
)


In [ ]:
# Run the Crew

crew = Crew(
    agents = [researcher_1, researcher_2, writer],
    tasks = [tasks[0], tasks[1], tasks[2]],
    manager_agent = manager,
    process = Process.hierarchical
)

result = crew.kickoff(
    inputs = {
        'topic': 'Autonomous Robotics',
        'year': 2025})

print(result)

In [47]:
crew = Crew(
    agents = [researcher_calculator, writer],
    tasks = [math_task],
    manager_agent = manager,
    process = Process.hierarchical
)

result = crew.kickoff(
    inputs = {
        'topic': 'Math operations',
        'operation': '2 + 56 / 22'})

print(result)

╭─────────────────────────────────────────────── Execution Traces ────────────────────────────────────────────────╮
│                                                                                                                 │
│  🔍 Detailed execution traces are available!                                                                    │
│                                                                                                                 │
│  View insights including:                                                                                       │
│    • Agent decision-making process                                                                              │
│    • Task execution flow and timing                                                                             │
│    • Tool usage details                                                                                         │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

The calculation of 56 / 22 is approximately 2.545.

Therefore, 2 + 56 / 22 = 2 + 2.545 = 4.545.

A brief report with the specific calculated growth figures:

The calculated growth figure for the operation 2 + 56 / 22 is approximately 4.545.Would you like to view your execution traces? [y/N] (20s timeout): 


╭────────────────────────── Tracing Preference Saved ──────────────────────────╮
│                                                                              │
│  Info: Tracing has been disabled.                                            │
│                                                                              │
│  Your preference has been saved. Future Crew/Flow executions will not        │
│  collect traces.                                                             │
│                                                                              │
│  To enable tracing later, do any one of these:                               │
│  • Set tracing=True in your Crew/Flow co